## NEMD: Shear Viscosity of Linear Alkanes

For our final workshop session we will be performing non-equilibrium molecular dynamics (NEMD) simulations of bulk systems of linear alkanes. Specifically, we will be using the SLLOD equations of motion to estimate shear viscosity.

**References:**

R. K. Bhupathiraju, S. T. Cui, S. A. Gupta, H. D. Cochran and P. T. Cummings, "Molecular Simulation of Rheological Properties using Massively Parallel Supercomputers," Proceedings of the 1996 ACM/IEEE Conference on Supercomputing, 1996, pp. 52-52.

P. J. Daivis and B. D. Todd, "A simple, direct derivation and proof of the validity of the SLLOD equations of motion for generalized homogeneous flows," JCP, 2006, _124_, 194103

### SLLOD Equations of Motion

Recall from lecture:

<img src="utils/sllod.png" alt="SLLOD" width="650px"/>

Derived by Evans and Morriss, the SLLOD equations of motion provide an approach for performing NEMD of homogenous planar shear flow.

\begin{align}
\dot{\boldsymbol{q_i}} & = \frac{\boldsymbol{p_i}}{m_i} + \boldsymbol{q_i} \cdot \nabla \boldsymbol{u} \\
\dot{\boldsymbol{p_i}} & = \boldsymbol{F_i} - \boldsymbol{p_i} \cdot \nabla \boldsymbol{u} \\
\end{align}

\begin{align}
m_i,\: \boldsymbol{q_i},\: \boldsymbol{p_i},\: \boldsymbol{F_i} = \mathrm{mass,\: position,\: momentum,\: and\: force\: on\: particle\: i} 
\end{align}

\begin{align}
\nabla \boldsymbol{u} = \mathrm{velocity\: gradient\: tensor}
\end{align}

### Construct the molecular system

We will again use the MoSDeF toolkit to build our system.

Recall on Wednesday we created an mBuild class for a system of bulk alkanes. This class has been included in a `utils` directory for us to reuse in this session. All we need to do is import the class.

In [ ]:
from utils import AlkaneBox

### Create a box of linear alkanes

With our class imported, we can create our system by instantiating and specifying the alkane chain length, the number of chains to place in the box, and the density at which we want to pack the box.

We don't want to create _too_ big of a system, so let's just use 100 chains. Concerning the chain length, you may choose any of the following, just be sure to use the correct density!

| Chain Length (number of carbons) | Density (kg/m^3) |
| --- | --- |
| 5 (pentane) | 626 |
| 6 (hexane) | 659 |
| 8 (octane) | 703 |
| 10 (decane) | 730 |

In [ ]:
chain_length = 8

alkane_box = AlkaneBox(chain_length=chain_length, n_chains=100, density=703)

Let's take a quick look at our system.

In [ ]:
alkane_box.visualize()

#### LAMMPS

We will be running our simulations today using the LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) simulation code. This simulation package dates back to the mid-1990s and is developed primarily at Sandia National Laboratory. Although LAMMPS does not approach the performance of GROMACS (due to a lack of rigorous GPU optimization), it remains one of the most popular simulation codes as it provides perhaps the greatest flexibility of any open-source molecular dynamics code. 

#### Saving to LAMMPS data format

mBuild can save `Compound` information directly to the data format required for LAMMPS. As with our systems on Wednesday, we will need to pass a force field to apply when saving so that all the necessary information is provided in our data file to run the simulation. The standard OPLS force field (which we used for our bulk alkane systems on Wednesday) has been shown by Allen and Rowley to overestimate the dynamic viscosity of linear alkanes. However, they showed that by reducing the value of the hydrogen σ from 2.5Å to 2.38Å the agreement was significantly improved. Thus, we will use the Allen-Rowley modification to the OPLS force field for our system.

In [ ]:
alkane_box.save('alkanes.lammps', forcefield_files='opls-ar.xml', overwrite=True,
                residues='Alkane')

The LAMMPS data file contains coordinate, topological, and force field information. Let's take a quick look at the beginning of the file.

In [ ]:
!head -n 50 pentane.lammps

### Running an NEMD simulation

Let's take a peek at the simulation input script.

In [1]:
!cat in.nemd

# Define variables
variable        temperature equal 298.15
variable	    srate equal ${STRAINRATE}
variable        seed equal 12345

# Use real units (Distance: angstroms, Time: femtoseconds, Energy: kcal/mol)
units		    real

# Define atom style (Full: bonds, angles, dihedrals, and charges)
atom_style	    full

# Define neighborlist params
neighbor        2.5 bin
neigh_modify    delay 5 every 1

# Define potential functional forms
pair_style      lj/cut/coul/long 11.0
bond_style      harmonic
angle_style     harmonic
dihedral_style  opls

# Set scaling of nonbonded interactions for 1-4 neighbors to 0.5
special_bonds   lj/coul 0 0 0.5

# Load system
read_data       alkanes.lammps

# Shift the LJ potential to 0 at the cutoff
pair_modify     shift yes

# Convert box to triclinic (allows us to deform the box during NEMD stage)
change_box      all triclinic

# Setup long-range electrostatics
kspace_style    pppm 1.0e-4

# Initialize velocities
velocity        all create ${temperature} ${se

Wow! That's a bit much to take in all at once. Let's break it down section-by-section.

In [3]:
!sed -n '1,5 p' in.nemd

# Define variables
variable        temperature equal 298.15
variable	    srate equal ${STRAINRATE}
variable        seed equal 12345



In the first few lines of our simulation script we define some variables that we will be using through the script, specifically the temperature (298.15K), strain rate (which we are reading in from the command line), and a seed for the random number generator (12345).

In [4]:
!sed -n '6,10 p' in.nemd

# Use real units (Distance: angstroms, Time: femtoseconds, Energy: kcal/mol)
units		    real

# Define atom style (Full: bonds, angles, dihedrals, and charges)
atom_style	    full


We need to tell LAMMPS what units to expect (and what to output). We'll choose **`real`** units, which means that _distances_ will be in **Angstroms**, _time_ will be in **femtoseconds**, and _energy_ will be in **kcal/mol**.

Additionally, we need to tell LAMMPS what format our data file will be in through the `atom_style` command. By choosing **`full`** we're telling LAMMPS to expect bond, angle, and dihedral information in addition to atom information, and also to read in partial charges for each atom.

In [5]:
!sed -n '12,14 p' in.nemd

# Define neighborlist params
neighbor        2.5 bin
neigh_modify    delay 5 every 1


Here, we define our neighborlist parameters. We tell LAMMPS to use a skin distance of 2.5Å, to use the `bin` style to generate the neighborlist, and to rebuild the neighborlist every 5 timesteps.

In [11]:
!sed -n '16,24 p; 28,30 p; 34,35 p' in.nemd

# Define potential functional forms
pair_style      lj/cut/coul/long 11.0
bond_style      harmonic
angle_style     harmonic
dihedral_style  opls

# Set scaling of nonbonded interactions for 1-4 neighbors to 0.5
special_bonds   lj/coul 0 0 0.5

# Shift the LJ potential to 0 at the cutoff
pair_modify     shift yes

# Setup long-range electrostatics
kspace_style    pppm 1.0e-4


Here we define our force field. The force field parameters themselves are read in from the data file, but we still need to tell LAMMPS which functional forms to use. We're going to be using a modified form of the OPLS force field, therefore

In [ ]:
!sed -n '18,21 p' in.nemd

In [ ]:
!sed -n '24p' in.nemd

In [ ]:
!sed -n '27p' in.nemd

In [ ]:
!sed -n '30p' in.nemd

In [ ]:
!sed -n '33p' in.nemd

In [ ]:
!sed -n '36p' in.nemd

In [ ]:
!sed -n '39p' in.nemd

In [ ]:
!sed -n '42p' in.nemd

In [ ]:
!sed -n '45p' in.nemd

In [ ]:
!sed -n '52,53 p' in.nemd

In [ ]:
!sed -n '56p' in.nemd

In [ ]:
!sed -n '58,62 p' in.nemd

In [ ]:
!sed -n '65p' in.nemd

In [ ]:
!sed -n '68,70 p' in.nemd

In [ ]:
!sed -n '77p' in.nemd

In [ ]:
!sed -n '80,81 p' in.nemd

In [ ]:
!sed -n '83,87 p' in.nemd

In [ ]:
!sed -n '90p' in.nemd

In [ ]:
!sed -n '92,96 p' in.nemd

In [ ]:
!sed -n '99,100 p' in.nemd

In [ ]:
!sed -n '103p' in.nemd

Before performing our simulation we need to choose a strain rate. Higher strain rates increase the signal-to-noise ratio and reach steady-state in a shorter amount of time; however, the viscosity of alkanes has been shown to decrease if the strain rate becomes too high:


From the above results, we see that the value of the strain rate where the viscosity begins to become underestimated is dependent on the chain length, with longer chains realizing these effects at lower strain rates. Based on these results, a strain rate of 0.01 should be sufficient to yield a reliable viscosity estimate for alkanes with ≤ 10 backbone carbons. We will pass this as an argument to the execution of the LAMMPS script in the command below.

**NOTE:** The command below was tested on an 8 core machine. Depending on your machine, you may need to alter the `mpirun` syntax and/or the number of processes spawned.

In [ ]:
!mpirun -np 8 lmp_mpi -in in.nemd -log nemd.log -var STRAINRATE 0.00001

### Comparing with experiment

| Chain Length (number of carbons) | Viscosity, Expt. (cP) | Viscosity, NEMD (cP) |
| --- | --- | --- |
| 5 (pentane) | 0.240 | ??? |
| 6 (hexane) | 0.3 | ??? |
| 8 (octane) | 0.542 | ??? |
| 10 (decane) | 0.920 | ??? |

The simulation should have generated a `nemd.log` output file. Here, we'll read that in to obtain a time series for the viscosity. We will use `matplotlib` to plot the viscosity time series alongside the expected value.

Execute the code below. How well did we do? Viscosity can be a difficult property to match with simulation, particularly if the force field was parameterized against only thermodynamic data. Results within 10-20% are acceptable.

**NOTE:** If your simulation has not completed, pre-computed results are available in the `data` directory. You can access this by changing the `filename` variable in the code below to either `data/pentane.log`, `data/hexane.log`, `data/octane.log`, or `data/decane.log` depending on the system you created above.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

filename = 'nemd.log'

expt = {5: 0.240, 6: 0.3, 8: 0.542, 10: 0.920}

time = []        # in ns
viscosity = []   # in cP

with open(filename, 'r') as f:
    for i, line in enumerate(f):
        if (i > 790 and len(line.split()) == 9 and 'SHAKE' not in line
            and 'Total' not in line):
            data = line.split()
            current_timestep = int(data[0])
            if not time:
                initial_timestep = current_timestep
            time.append((current_timestep - initial_timestep) * 2 / 1e6)
            viscosity.append(float(data[-1]) * 101325 / 10**15 * 1000)

fig, ax = plt.subplots()

# Plot measured viscosity
ax.plot(time, viscosity, label='NEMD')

# Plot expected value
expt_val = expt[chain_length]
xdata = np.linspace(time[0], time[-1], 100)
ydata = [expt_val for _ in xdata]
plt.plot(xdata, ydata, linestyle='--', color='k', label='Expected')

ax.set_xlabel('Time, ns')
ax.set_ylabel('Viscosity, cP')
ax.legend()

plt.show()